# Basi di Dati Mod. 2 - SQLAlchemy ORM

### Stefano Calzavara, Università Ca' Foscari Venezia

Gli ORM forniscono uno strumento per definire un mapping fra:
1. classi del linguaggio di programmazione (es. Python) e tabelle del DBMS sottostante
2. istanze di tali classi (oggetti) e righe delle tabelle corrispondenti

Il risultato è un sistema che sincronizza trasparentemente tutti i cambiamenti di stato fra gli oggetti e le righe ad essi associate. Tale approccio permette inoltre di esprimere query al database in termini di classi e delle relazioni esistenti fra di esse.

![alt text](orm.jpg "ORM architecture")

SQLAlchemy ORM è costruito sopra all'Expression Language ed offre un livello di astrazione ancora più elevato rispetto al DBMS sottostante. La maggior parte delle applicazioni può fare uso esclusivo delle funzionalità ORM, usando l'Expression Language solo dove è veramente necessario scendere a più basso livello.

### Dichiarazione di un mapping

In [ ]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite://', echo=True)

Base = declarative_base()                      # tabella = classe che eredita da Base

class User(Base):
    __tablename__ = 'users'                   # obbligatorio

    id = Column(Integer, primary_key=True)    # almeno un attributo deve fare parte della primary key
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    # questo metodo è opzionale, serve solo per pretty printing
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)

### Creare uno schema

In [ ]:
User.__table__

In [ ]:
Base.metadata.create_all(engine)

### Creare un'istanza di una classe mappata

Un'istanza di una classe mappata rappresenta una riga della tabella corrispondente.

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='thunder')
print(ed_user.name)
print(ed_user.nickname)
print(ed_user.id)

Importante: Si noti che a questo punto non è ancora stato scritto niente nel database! Questo è confermato dal fatto che l'attributo `id` è ancora impostato a `None`. Abbiamo però preparato SQLAlchemy ORM affinché sia in grado di aggiornare il database correttamente all'interno di una **sessione**.

### Sessioni: creazione ed utilizzo

Una sessione in SQLAlchemy ORM nasconde una serie di dettagli implementativi che tipicamente sono gestiti manualmente nell'Expression Language, in particolare la gestione delle connessioni e delle transazioni.

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)       # factory pattern
session = Session()

In [ ]:
session.add(ed_user)    # pending instance: verrà salvata nel database quando veramente necessario

In [ ]:
our_user = session.query(User).filter_by(name='ed').first()    # qui è necessario salvare la pending instance
our_user

In [ ]:
ed_user is our_user

In [ ]:
print(ed_user.id)    # primary key creata in fase di scrittura al database

In [ ]:
ed_user.nickname = 'eddie'

session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy'),
                 User(name='mary', fullname='Mary Contrary', nickname='mary'),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [ ]:
print("Dirty instances: " + str(session.dirty))
print("Pending instances: " + str(session.new) + "\n")
session.commit()
print("\nDirty instances: " + str(session.dirty))
print("Pending instances: " + str(session.new))

In [ ]:
ed_user.name = 'Edwardo'
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

In [ ]:
session.rollback()
print(">>> eddie's name: " + ed_user.name)
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

### Selezionare dati

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

In [ ]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

In [ ]:
for u in session.query(User).filter_by(fullname='Ed Jones'):
    print(u.name)

In [ ]:
for u in session.query(User).filter(User.fullname=='Ed Jones'):
    print(u.name)

In [ ]:
for u in session.query(User).filter(User.name.like('%ed')).filter(User.fullname=='Ed Jones'):
    print(u.name)

In [ ]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

In [ ]:
query.first()

In [ ]:
query.one()

In [ ]:
query.count()

### Relazioni

Modelliamo ora una relazione uno-a-molti fra utenti ed indirizzi email. Tale relazione consente di effettuare le seguenti operazioni:
1. Dato un indirizzo email, trovare l'utente ad esso associato
2. Dato un utente, trovare una lista dei suoi indirizzi email

Questo meccanismo viene implementato tramite `relationship` come segue. Si faccia attenzione alla creazione di due attributi in questo caso, uno per ciascuna classe. Il primo attributo implementa 1, mentre il secondo attributo implementa 2.

In [ ]:
from sqlalchemy.orm import relationship

class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey(User.id))

     user = relationship(User, back_populates="addresses")    # qui viene sfruttata la foreign key

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address
        
User.addresses = relationship(Address, order_by=Address.id, back_populates="user")

In [ ]:
Base.metadata.create_all(engine)

jack = User(name='jack', fullname='Jack Bean', nickname='j&b')
jack.addresses

In [ ]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [ ]:
jack.addresses[0]

In [ ]:
jack.addresses[0].user

In [ ]:
session.add(jack)
session.commit()

In [ ]:
jack = session.query(User).filter_by(name='jack').one()
print(jack)     # nota: nessuna istruzione SQL viene eseguita per la tabella addresses

In [ ]:
jack.addresses   # solo a questo punto viene eseguito SQL per la tabella addresses

### Giunzioni

In [ ]:
for u, a in session.query(User, Address):
    print("({}, {})".format(u,a))

In [ ]:
for u, a in session.query(User, Address).filter(User.id == Address.user_id):
    print("({}, {})".format(u,a))

In [ ]:
session.query(User).join(Address).count()

In [ ]:
# https://docs.sqlalchemy.org/en/13/faq/sessions.html#faq-query-deduplicating
session.query(User).join(Address).all()

In [ ]:
session.query(User.name, User.fullname, User.nickname).join(Address).all() 

In [ ]:
session.query(User, Address.email_address).join(Address).all()

In [ ]:
session.query(User, Address.email_address).outerjoin(User.addresses).all()

### Cancellazioni

In [ ]:
session.delete(jack)
print("Deleted instances: " + str(session.deleted))
print(session.query(User).filter(User.name == 'jack').count())
print("Deleted instances: " + str(session.deleted))

In [ ]:
session.query(Address).all()    # nessuna forma di cascading!

In [ ]:
session.rollback()

Base = declarative_base()

class User(Base):
     __tablename__ = 'users'                   # obbligatorio

     id = Column(Integer, primary_key=True)    # almeno un attributo deve fare parte della primary key
     name = Column(String)
     fullname = Column(String)
     nickname = Column(String)
     
     # configuriamo la politica di cascading
     addresses = relationship("Address", back_populates='user', cascade="all, delete, delete-orphan")

     def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)
    
class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey(User.id))

     user = relationship(User, back_populates="addresses")

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address

In [ ]:
jack = session.query(User).filter(User.name == 'jack').first()

for a in jack.addresses:
    print(a)

In [ ]:
del jack.addresses[1]
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

In [ ]:
session.delete(jack)
print(session.query(User).filter(User.name == 'jack').count())
print(session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count())

### Relazioni molti-a-molti

Modelliamo infine una relazione molti-a-molti fra blog post e keywords al loro interno. Ciò richiede la creazione di una **tabella di associazione**, che sporca l'eleganza della rappresentazione ORM vista fino ad ora. Tale tabella deve avere esattamente due colonne, che operano da chiavi esterne verso le due relazioni da associare.

Tramite la tabella di associazione è possibile effettuare le seguenti operazioni:
1. Dato un blog post, trovare la lista delle sue keywords
2. Data una keyword, trovare la lista dei blog post in cui occorre

Sebbene siamo costretti ad esporre tale dettaglio implementativo, l'interfaccia di accesso alle informazioni desiderate tramite l'ORM rimane comunque estremamente semplice. E' possibile estendere tale pattern a strutture con più di due colonne usando **oggetti di associazione**: per i dettagli potete consultare la documentazione ufficiale [qui](https://docs.sqlalchemy.org/en/13/orm/basic_relationships.html#relationship-patterns).

In [ ]:
from sqlalchemy import Table

post_keywords = Table('post_keywords', Base.metadata,
                                       Column('post_id', ForeignKey('posts.id'), primary_key=True),
                                       Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [ ]:
class BlogPost(Base):
    __tablename__ = 'posts'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)

    # relazione molti-a-molti
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')

    def __repr__(self):
        return "BlogPost(%r, %r, %r)" % (self.headline, self.body, self.author)
    
class Keyword(Base):
    __tablename__ = 'keywords'

    id = Column(Integer, primary_key=True)
    kw = Column(String(50), nullable=False, unique=True)
    
    # relazione molti-a-molti
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    # costruttore esplicito (opzionale)
    def __init__(self, kw):
        self.kw = kw

Aggiungiamo poi una relazione uno-a-molti fra utenti e blog post, riutilizzando le tecniche già viste. Si noti che avevamo già introdotto una chiave esterna su `BlogPost` per questo compito.

In [ ]:
BlogPost.author = relationship(User, back_populates="posts")
User.posts = relationship(BlogPost, back_populates="author")

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
wendy = session.query(User).filter_by(name='wendy').one()
post = BlogPost(headline="Wendy's Blog Post", body="This is a test", author=wendy)
session.add(post)

In [ ]:
post.keywords = [Keyword('wendy'), Keyword('firstpost')]

In [ ]:
session.query(BlogPost).filter(BlogPost.keywords.any(kw='firstpost')).all()

In [ ]:
session.query(BlogPost).filter(BlogPost.author == wendy).all()

In [ ]:
wendy.posts

## Esercizio

Creare tramite SQLAlchemy ORM le seguenti tabelle:
* Product(maker, model*, type)
* PC(<u>model*</u>, speed, ram, hd, price)
* Laptop(<u>model*</u>, speed, ram, hd, screen, price)
* Printer(<u>model*</u>, color, type, price)

Definite opportune chiavi primarie ed esterne, scegliendo i tipi di dato appropriati per i vari attributi. Una volta fatto ciò, popolate le tabelle con alcuni dati artificiali ed effettuate tramite l'ORM di SQLAlchemy le seguenti query:
1. Trovare il modello, la velocità e la dimensione dell'hard disk di tutti i PC che costano meno di $1000. 
2. Trovare tutti i produttori di stampanti.
3. Trovare il produttore e la velocità dei laptop con un hard disk da almeno 300 GB.
4. Trovare il modello ed il prezzo di tutti i PC ed i laptop realizzati dalla Lenovo.
5. Trovare le dimensioni degli hard disk che occorrono in almeno due PC.
6. Trovare tutte le aziende che producono laptop, ma non PC.
7. Trovare i produttori di PC con una velocità minima di 2.0 GHz.
8. Trovare tutte le aziende che producono sia PC che laptop.

Procedete immaginando la query SQL e traducendola nell'ORM di SQLAlchemy. Consultate la documentazione dove necessario.